
# Space

In [ ]:
import os
import sys 
import logging
import pandas as pd 
from pprint import pprint 

# WorkSpace
KEY = 'WorkSpace'; WORKSPACE_PATH = os.getcwd().split(KEY)[0] + KEY; print(WORKSPACE_PATH)
os.chdir(WORKSPACE_PATH)
sys.path.append(WORKSPACE_PATH)

# Pipeline Space
from proj_space import SPACE
SPACE['WORKSPACE_PATH'] = WORKSPACE_PATH
sys.path.append(SPACE['CODE_FN'])
pprint(SPACE)

# Available Packages
import pandas as pd
from datetime import datetime 

logger = logging.getLogger(__name__)
recfldtkn_config_path = os.path.join(SPACE['CODE_FN'], 'config_recfldtkn/')


# Trigger Events

In [ ]:
import datasets
from recfldtkn.loadtools import load_ds_rec_and_info
from recfldtkn.configfn import load_cohort_args, load_record_args
from recfldtkn.loadtools import convert_variables_to_pystirng, load_module_variables

base_config = load_cohort_args(recfldtkn_config_path, SPACE)
print(base_config)

In [ ]:
# CaseFolder = os.path.join(SPACE['DATA_TASK'], 'CaseFolder')
# print(CaseFolder)

# Trigger Case

In [ ]:
#################
cohort_label_list = [1, 2, 3]
#################

######################################
TriggerCaseMethod = 'CGM5MinEntry'
######################################

TriggerRecName = 'CGM5Min'
case_id_columns = ["PID", "ObsDT"]
special_columns = ['PID', 'DT_s']

In [ ]:

import inspect

def convert_TriggerEvent_to_Caseset(ds_rec, case_id_columns, special_columns, base_config):
    ds_case = ds_rec.select_columns(special_columns)
    df_case_raw = ds_case.to_pandas()
    
    # ------------------------------ Filter Cases ------------------------------
    df_case_filter = df_case_raw # [df_case_raw['invitation_state'] == 'Delivered'].reset_index(drop=True)
    # --------------------------------------------------------------------------

    # ------------------------------ Determine the ObsDT ------------------------------
    df_case_filter['ObsDT'] = df_case_filter['DT_s'] # [df_case_raw['drug_name'] == 'Trulicity']
    df_case_filter = df_case_filter.drop(columns = 'DT_s') # .from_pandas(df_case_filter)
    # --------------------------------------------------------------------------

    # ------------------------------- Update Column Sequence ------------------------
    columns = df_case_filter.columns 
    columns = case_id_columns + [col for col in columns if col not in case_id_columns]
    df_case_filter = df_case_filter[columns].reset_index(drop=True)

    df_case = df_case_filter
    return df_case

convert_TriggerEvent_to_Caseset.fn_string = inspect.getsource(convert_TriggerEvent_to_Caseset)

In [ ]:
prefix = ['import pandas as pd', 'import numpy as np']
string_variables = [TriggerRecName]
iterative_variables = [case_id_columns, special_columns]
fn_variables = [convert_TriggerEvent_to_Caseset]
pycode = convert_variables_to_pystirng(string_variables = string_variables, 
                                       iterative_variables = iterative_variables, 
                                       fn_variables = fn_variables, prefix = prefix)
pypath = os.path.join(base_config['trigger_pyfolder'], f'{TriggerCaseMethod}.py')
print(pypath)
if not os.path.exists(os.path.dirname(pypath)): os.makedirs(os.path.dirname(pypath))
with open(pypath, 'w') as file: file.write(pycode)

In [ ]:
pypath = os.path.join(base_config['trigger_pyfolder'], f'{TriggerCaseMethod}.py')
module = load_module_variables(pypath)

######################
TriggerRecName = module.TriggerRecName
case_id_columns = module.case_id_columns
special_columns = module.special_columns
convert_TriggerEvent_to_Caseset = module.convert_TriggerEvent_to_Caseset
######################

In [ ]:
############################### ds_rec to df_case ###############################
ds_rec, _ = load_ds_rec_and_info(TriggerRecName, base_config, cohort_label_list)
df_case = convert_TriggerEvent_to_Caseset(ds_rec, case_id_columns, special_columns, base_config)
df_case

In [ ]:
 #  = [1, 2, 3]

In [ ]:
CaseFolder = os.path.join(SPACE['DATA_CaseSet'])

if not os.path.exists(CaseFolder): os.makedirs(CaseFolder)

cohort_label_list_str = '_'.join([str(i) for i in cohort_label_list])
# path = os.path.join(CaseFolder, f'{TriggerCaseMethod}-{cohort_label_list_str}.p')
path = os.path.join(CaseFolder, f'{TriggerCaseMethod}.p')
df_case.to_pickle(path)
print(path)